In [4]:
# %load naive_bayes.py
import numpy as np

class NaiveBayesBinaryClassifier:
    
    def fit(self, X, y):
        self.y_classes, y_counts = np.unique(y, return_counts=True)
        self.phi_y = 1.0 * y_counts/y_counts.sum()
        self.phi_x = [1.0 * X[y==k].mean(axis=0) for k in self.y_classes]
        return self
    
    def predict(self, X):
        return np.apply_along_axis(lambda x: self.compute_probs(x), 1, X)
    
    def compute_probs(self, x):
        probs = np.array([self.compute_prob(x, y) for y in range(len(self.y_classes))])
        return self.y_classes[np.argmax(probs)]
    
    def compute_prob(self, x, y):
        res = 1
        for j in range(len(x)):
            Pxy = self.phi_x[y][j] # p(xj=1|y)
            res *= (Pxy**x[j])*((1-Pxy)**(1-x[j])) # p(xj=0|y)
        return res * self.phi_y[y]
    
    def evaluate(self, X, y):
        return (self.predict(X) == y).mean()
    
class MultinomialNB:
    
    def fit(self, X, y):
        self.y_classes, y_counts = np.unique(y, return_counts=True)
        self.x_classes = np.array([np.unique(x) for x in X.T])
        self.phi_y = 1.0 * y_counts/y_counts.sum()
        self.phi_x = self.mean_x(X, y)
        return self
    
    def mean_x(self, X, y):
        return [[(X[:,j][y==k].reshape(-1,1) == self.x_classes[j]).mean(axis=0)
                       for j in range(len(self.x_classes))]
                      for k in self.y_classes]
    
    def predict(self, X):
        return np.apply_along_axis(lambda x: self.compute_probs(x), 1, X)
    
    def compute_probs(self, x):
        probs = np.array([self.compute_prob(x, y) for y in range(len(self.y_classes))])
        return self.y_classes[np.argmax(probs)]
    
    def compute_prob(self, x, y):
        Pxy = 1
        for j in range(len(x)):
            i = list(self.x_classes[j]).index(x[j])
            Pxy *= self.phi_x[y][j][i] # p(xj|y)
        return Pxy * self.phi_y[y]
    
    def evaluate(self, X, y):
        return (self.predict(X) == y).mean()
    
    
class GaussianNB:
    
    def fit(self, X, y, epsilon = 1e-10):
        self.y_classes, y_counts = np.unique(y, return_counts=True)
        self.x_classes = np.array([np.unique(x) for x in X.T])
        self.phi_y = 1.0 * y_counts/y_counts.sum()
        self.u = np.array([X[y==k].mean(axis=0) for k in self.y_classes])
        self.var_x = np.array([X[y==k].var(axis=0)  + epsilon for k in self.y_classes])
        return self
    
    def predict(self, X):
        return np.apply_along_axis(lambda x: self.compute_probs(x), 1, X)
    
    def compute_probs(self, x):
        probs = np.array([self.compute_prob(x, y) for y in range(len(self.y_classes))])
        return self.y_classes[np.argmax(probs)]
    
    def compute_prob(self, x, y):
        c = 1.0 /np.sqrt(2.0 * np.pi * (self.var_x[y]))
        return np.prod(c * np.exp(-1.0 * np.square(x - self.u[y]) / (2.0 * self.var_x[y])))
    
    def evaluate(self, X, y):
        return (self.predict(X) == y).mean()
    

In [5]:
from sklearn import datasets
from utils import accuracy_score
iris = datasets.load_iris()
X = iris.data  
y = iris.target
MultinomialNB().fit(X, y).evaluate(X, y)

0.9666666666666667

In [72]:
X [0]

array([5.1, 3.5, 1.4, 0.2])

In [65]:
from sklearn import datasets
from utils import accuracy_score
iris = datasets.load_iris()
X = iris.data
y = iris.target
GaussianNB().fit(X, y).evaluate(X, y)

0.96

In [66]:
from sklearn import datasets
from utils import accuracy_score

iris = datasets.load_iris()
X = iris.data  
X = (X > X.mean(axis=0))*1 # turn our feature to categorical using mean
y = iris.target
NaiveBayesBinaryClassifier().fit(X, y).evaluate(X, y)

0.7533333333333333

In [67]:
X = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])
Y = np.array([1, 1, 1, 2, 2, 2])
clf = MultinomialNB().fit(X, Y)
print(clf.predict(X))

[1 1 1 2 2 2]


In [68]:
from sklearn import datasets
digits = datasets.load_digits()
X = digits.data
y = digits.target
MultinomialNB().fit(X, y).evaluate(X, y)

0.9833055091819699

In [69]:
from sklearn import datasets
digits = datasets.load_digits()
X = digits.data
y = digits.target
GaussianNB().fit(X, y).evaluate(X, y)

0.8091263216471898

In [70]:
b = np.array([0, 1])
a = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0])

In [61]:
from sklearn import datasets
from utils import accuracy_score, train_test_split
iris = datasets.load_iris()
X = iris.data
# X = (X-X.min(axis=0)).astype(int)
y = iris.target
X_tr, X_ts, y_tr, y_ts = train_test_split(X,y)
GaussianNB().fit(X_tr, y_tr).evaluate(X_ts, y_ts)

0.9666666666666667

### Naive Bayes

In [40]:
X_tr[:10]

array([[0, 1, 0, 0],
       [2, 1, 3, 1],
       [2, 1, 3, 1],
       [0, 1, 0, 0],
       [1, 0, 3, 1],
       [2, 0, 3, 1],
       [2, 1, 4, 2],
       [2, 1, 3, 1],
       [1, 0, 3, 1],
       [2, 1, 5, 2]])

In [41]:
y_classes = [0 1 2]
P(y) = []

array([0, 1, 1, 0, 1, 1, 2, 1, 1, 2])

In [ ]:
p(y|x) = p(x|y) * p(y)
x_uniques = [0 1 2]

In [ ]:
p(y|x) = p(x|y) * p(y) / p(x)

In [ ]:
p(x|y) 

In [ ]:
p(y) = [0.2, 0.6, 0.2]

In [ ]:
p(x0|y=0) = [1 0 0]
p(x0|y=1) = [0 0.33 0.6]
p(x0|y=2) = [0 0.33 0.6]